In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common import NoSuchElementException
from selenium.common import TimeoutException
import numpy as np
from selenium.webdriver import ActionChains
import pandas as pd

In [2]:
url='https://www.google.com/search?rlz=1C1GCEA_enHR1085HR1085&q=Casablanca%20Reviews&rflfq=1&num=20&stick=H4sIAAAAAAAAAONgkxI2sTQzMjUzsDC2NDUzMTMzN7W03MDI-IpRyDmxODEpJzEvOVEhKLUsM7W8eBErFkEAULFNy0QAAAA&rldimm=4962560839564667599&tbm=lcl&hl=en&sa=X&ved=0CAYQ5foLahcKEwiAzYbJzIKEAxUAAAAAHQAAAAAQCg&biw=1536&bih=729&dpr=1.25#lkt=LocalPoiReviews&arid=ChZDSUhNMG9nS0VJQ0FnSURSNWJlRUhnEAE'

In [3]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

driver.get(url)

AcceptCookies = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept all')]")))
AcceptCookies.click()
driver.maximize_window()

In [13]:
driver.quit()


In [4]:
ReviewsTab = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'SVWlSe') and contains(@class, 't35a5d')]//span[text()='Reviews']")))
ReviewsTab.click()

In [5]:
total_reviews_element=driver.find_element(By.XPATH, "//*[@id='akp_tsuid_13']/div/div[1]/div/g-sticky-content-container/div/block-component/div/div[1]/div/div/div/div[1]/div/div/div[6]/g-flippy-carousel/div/div/ol/li[3]/span/div/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/span/span")
total_reviews_text = total_reviews_element.text
total_reviews = int(total_reviews_text.split()[0])

In [ ]:
%%sql


In [9]:

while loaded_reviews_count < total_reviews:
    loaded_reviews_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")
    for review_element in loaded_reviews_elements[loaded_reviews_count:]:
        reviews.append(review_element.text)

    loaded_reviews_count = len(loaded_reviews_elements)

    if loaded_reviews_count < total_reviews:
        driver.execute_script("arguments[0].scrollIntoView();", loaded_reviews_elements[-1])
        time.sleep(5)

KeyboardInterrupt: 

In [ ]:
reviews_df = pd.DataFrame(reviews, columns=['Review'])

In [41]:
loaded_reviews_count = 0
reviews = []

# Load Reviews Until Empty Ones are Encountered
while True:
    review_elements = driver.find_elements(By.XPATH, "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div")
    if not review_elements:
        break

    for index, element in enumerate(review_elements, start=1):
        current_review_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{index}]//div[@class='OA1nbd']"
        current_review = driver.find_element(By.XPATH, current_review_xpath)

        # Check for empty review
        review_text = current_review.text.strip()
        if not review_text:
            break

        # Click Translate if Available
        translate_button = current_review.find_elements(By.XPATH, ".//button[contains(text(), 'Translated by Google')]")
        if translate_button:
            translate_button[0].location_once_scrolled_into_view
            translate_button[0].click()
            time.sleep(1)

        # Click More if Available
        more_button = current_review.find_elements(By.XPATH, ".//a[@role='button' and contains(text(), 'More')]")
        if more_button:
            more_button[0].location_once_scrolled_into_view
            more_button[0].click()
            time.sleep(1)

        # Add Review Text to Array
        if review_text:
            reviews.append(review_text)

        loaded_reviews_count += 1

    # Scroll to the last element
    last_element = review_elements[-1]
    last_element.location_once_scrolled_into_view
    time.sleep(2)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[24]//div[@class='OA1nbd']"}
  (Session info: chrome=121.0.6167.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7EC375E42+3538674]
	(No symbol) [0x00007FF7EBF94C02]
	(No symbol) [0x00007FF7EBE45AEB]
	(No symbol) [0x00007FF7EBE8BF4E]
	(No symbol) [0x00007FF7EBE8C0CC]
	(No symbol) [0x00007FF7EBECE477]
	(No symbol) [0x00007FF7EBEAF0EF]
	(No symbol) [0x00007FF7EBECBDE3]
	(No symbol) [0x00007FF7EBEAEE53]
	(No symbol) [0x00007FF7EBE7F514]
	(No symbol) [0x00007FF7EBE80631]
	GetHandleVerifier [0x00007FF7EC3A6CAD+3738973]
	GetHandleVerifier [0x00007FF7EC3FC506+4089270]
	GetHandleVerifier [0x00007FF7EC3F4823+4057299]
	GetHandleVerifier [0x00007FF7EC0C5C49+720121]
	(No symbol) [0x00007FF7EBFA12

In [61]:
df_reviews= pd.DataFrame(reviews, columns=['Review'])

In [62]:
df_reviews.to_csv('reviews.csv', index=False)


In [48]:
# Load Reviews Until Empty Ones are Encountered
# Load Reviews Until Empty Ones are Encountered
while True:
    review_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")
    if review_elements:
        last_element = review_elements[-1]
        last_element.location_once_scrolled_into_view
        time.sleep(2)
        if not last_element.text.strip():
            break

# Scroll Back to First Review
first_review = driver.find_element(By.XPATH, f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{1}]//div[@class='OA1nbd']")
first_review.location_once_scrolled_into_view
time.sleep(2)

# Process Each Review
reviews = []
for i in range(len(review_elements)):
    # Fetch the review element again to avoid stale element reference
    current_review = driver.find_elements(By.XPATH, f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[@class='OA1nbd']")

    # Click Translate if Available
    translate_button = current_review.find_elements(By.XPATH, ".//button[contains(text(), 'Translated by Google')]")
    if translate_button:
        translate_button[0].location_once_scrolled_into_view
        translate_button[0].click()
        time.sleep(1)

    # Click More if Available
    more_button = current_review.find_elements(By.XPATH, ".//a[@role='button' and contains(text(), 'More')]")
    if more_button:
        more_button[0].location_once_scrolled_into_view
        more_button[0].click()
        time.sleep(1)

    # Re-fetch the current review to get the updated text
    current_review = driver.find_elements(By.XPATH, f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[@class='OA1nbd']")
    review_text = current_review.text.strip()
    if review_text:
        reviews.append(review_text)

    # Scroll to Next Review, if not the last review
    if i < len(review_elements) - 1:
        next_review = driver.find_elements(By.XPATH, f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i+1}]//div[@class='OA1nbd']")
        next_review.location_once_scrolled_into_view
        time.sleep(1)



KeyboardInterrupt: 

In [58]:
# Initialize a variable to store the text of the last review from the previous iteration
previous_last_review_text = None

while True:
    review_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")
    if not review_elements:
        break  # Break if no reviews are found

    last_element = review_elements[-1]
    last_element.location_once_scrolled_into_view
    time.sleep(2)  # Waiting for potential loading of new reviews

    current_last_review_text = last_element.text.strip()

    # Check if the last review text is the same as in the previous iteration
    if current_last_review_text == previous_last_review_text:
        break  # Break the loop if we are seeing the same last review as before

    previous_last_review_text = current_last_review_text  # Update the last review text for the next iteration

# Continue with the rest of your code...


In [59]:
first_review = driver.find_element(By.XPATH, "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[2]//div[@class='OA1nbd']")
first_review.location_once_scrolled_into_view
time.sleep(2)
driver.execute_script("arguments[0].scrollIntoView();", first_review)


In [60]:

# Scroll Back to First Review

# Process Each Review
reviews = []
for i in range(1, len(review_elements) + 1):
    # Fetch the review element again to avoid stale element reference
    current_review_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[@class='OA1nbd']"
    current_review = driver.find_element(By.XPATH, current_review_xpath)

    # Click Translate if Available
    translate_button = current_review.find_elements(By.XPATH, ".//button[contains(text(), 'Translated by Google')]")
    if translate_button:
        translate_button[0].location_once_scrolled_into_view
        translate_button[0].click()
        time.sleep(1)

    # Click More if Available
    more_button = current_review.find_elements(By.XPATH, ".//a[@role='button' and contains(text(), 'More')]")
    if more_button:
        more_button[0].location_once_scrolled_into_view
        more_button[0].click()
        time.sleep(1)

    # Re-fetch the current review to get the updated text
    current_review = driver.find_element(By.XPATH, current_review_xpath)
    review_text = current_review.text.strip()
    if review_text:
        reviews.append(review_text)

    # Scroll to Next Review, if not the last review
    if i < len(review_elements):
        next_review_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i+1}]//div[@class='OA1nbd']"
        next_review = driver.find_element(By.XPATH, next_review_xpath)
        next_review.location_once_scrolled_into_view
        time.sleep(1)


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=121.0.6167.85)
Stacktrace:
	GetHandleVerifier [0x00007FF7EC375E42+3538674]
	(No symbol) [0x00007FF7EBF94C02]
	(No symbol) [0x00007FF7EBE4599D]
	(No symbol) [0x00007FF7EBE8D406]
	(No symbol) [0x00007FF7EBE81950]
	(No symbol) [0x00007FF7EBEAF0AA]
	(No symbol) [0x00007FF7EBE8124A]
	(No symbol) [0x00007FF7EBEAF2C0]
	(No symbol) [0x00007FF7EBECBDE3]
	(No symbol) [0x00007FF7EBEAEE53]
	(No symbol) [0x00007FF7EBE7F514]
	(No symbol) [0x00007FF7EBE80631]
	GetHandleVerifier [0x00007FF7EC3A6CAD+3738973]
	GetHandleVerifier [0x00007FF7EC3FC506+4089270]
	GetHandleVerifier [0x00007FF7EC3F4823+4057299]
	GetHandleVerifier [0x00007FF7EC0C5C49+720121]
	(No symbol) [0x00007FF7EBFA126F]
	(No symbol) [0x00007FF7EBF9C304]
	(No symbol) [0x00007FF7EBF9C432]
	(No symbol) [0x00007FF7EBF8BD04]
	BaseThreadInitThunk [0x00007FFC09BE7344+20]
	RtlUserThreadStart [0x00007FFC0B0026B1+33]
